# Programm zur Erkennung von Hunderassen

## Schritt 1: Daten laden und Aufteilen in Trainings- und Testdaten

In [1]:
# Importieren der notwendigen Bibliotheken
import os
import numpy as np
import xml.etree.ElementTree as ET
from skimage import io, color, transform
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder


In [2]:
# Definieren der Pfade zum Datensatz und zu den Annotationen
dataset_path = 'dogs'
annotation_path = 'annotations'


In [3]:
# Funktion zum Parsen der Annotationsdateien
def parse_annotation(xml_file):
    tree = ET.parse(xml_file)
    root = tree.getroot()
    bndboxes = []
    for obj in root.findall('object'):
        bndbox = obj.find('bndbox')
        xmin = int(float(bndbox.find('xmin').text))
        ymin = int(float(bndbox.find('ymin').text))
        xmax = int(float(bndbox.find('xmax').text))
        ymax = int(float(bndbox.find('ymax').text))
        bndboxes.append((xmin, ymin, xmax, ymax))
    return bndboxes


In [4]:
# Laden der Bilder und Extrahieren der Hundebereiche
images = []
labels = []
breeds = os.listdir(dataset_path)

for breed in breeds:
    breed_img_path = os.path.join(dataset_path, breed)
    breed_anno_path = os.path.join(annotation_path, breed)
    if os.path.isdir(breed_img_path):
        for img_name in os.listdir(breed_img_path):
            img_path = os.path.join(breed_img_path, img_name)
            annotation_name = os.path.splitext(img_name)[0]
            annotation_file = os.path.join(breed_anno_path, annotation_name)
            if not os.path.exists(annotation_file):
                continue  # Überspringe Bilder ohne Annotation
            image = io.imread(img_path)
            if image is None:
                continue
            bndboxes = parse_annotation(annotation_file)
            for bndbox in bndboxes:
                xmin, ymin, xmax, ymax = bndbox
                xmin = max(0, xmin)
                ymin = max(0, ymin)
                xmax = min(image.shape[1], xmax)
                ymax = min(image.shape[0], ymax)
                if xmin >= xmax or ymin >= ymax:
                    continue  # Ungültige Bounding Box überspringen
                cropped_image = image[ymin:ymax, xmin:xmax]
                cropped_image = transform.resize(cropped_image, (256, 256))
                images.append(cropped_image)
                labels.append(breed)


In [5]:
# Umwandeln der Labels in numerische Werte
le = LabelEncoder()
y = le.fit_transform(labels)

# Überprüfen, ob Bilder geladen wurden
if len(images) == 0:
	print("Fehler: Keine Bilder geladen. Bitte überprüfen Sie den Pfad und die Annotationsdateien.")
else:
	# Aufteilen in Trainings- und Testdaten
	X_train_img, X_test_img, y_train, y_test = train_test_split(images, y, test_size=0.25, stratify=y, random_state=42)


In [6]:
# Überprüfen der Anzahl der geladenen Bilder
print(f'Anzahl der gesamten Bilder: {len(images)}')
print(f'Anzahl der Trainingsbilder: {len(X_train_img)}')
print(f'Anzahl der Testbilder: {len(X_test_img)}')


Anzahl der gesamten Bilder: 1029
Anzahl der Trainingsbilder: 771
Anzahl der Testbilder: 258


In [7]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam

# Definieren der Modellarchitektur
def build_model(input_shape, num_classes):
    model = Sequential()

    # Erstes Convolutional und MaxPooling Layer
    model.add(Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    # Zweites Convolutional und MaxPooling Layer
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    # Drittes Convolutional und MaxPooling Layer
    model.add(Conv2D(128, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    # Vierte Convolutional und MaxPooling Layer
    model.add(Conv2D(128, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    # Flatten Layer
    model.add(Flatten())

    # Volle Schicht mit Dropout für Regulierung
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.5))

    # Ausgabeschicht
    model.add(Dense(num_classes, activation='softmax'))

    return model

# Annahme: input_shape = (256, 256, 3) als Beispiel und num_classes musst du entsprechend deiner Daten setzen
input_shape = (256, 256, 3)
num_classes = len(np.unique(y_train))  # Anzahl der Hunderassen

# Modell erstellen
model = build_model(input_shape, num_classes)

# Modell kompilieren
# Wir werden die Loss-Funktion und den Optimizer im nächsten Schritt diskutieren und festlegen
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

C:\Users\OIJ1FE\AppData\Roaming\Python\Python310\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [8]:
# Modell kompilieren - dies ist bereits im vorherigen Schritt enthalten
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [10]:
from sklearn.model_selection import KFold
from tensorflow.keras.utils import to_categorical
import numpy as np

# Umwandeln der Zielwerte in One-Hot Kodierung (falls nötig)
y_train_categorical = to_categorical(y_train, num_classes=num_classes)

# Initialisierung der K-Fold Kreuzvalidierung
kfold = KFold(n_splits=3, shuffle=True, random_state=42)
fold_no = 1
scores = []

# Kreuzvalidierung
for train_idx, val_idx in kfold.split(X_train_img, y_train_categorical):
    # Modell erstellen
    model = build_model(input_shape, num_classes)
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    
    print(f'Training für fold {fold_no}...')
    
    # Fit Modell
    history = model.fit(np.array(X_train_img)[train_idx], np.array(y_train_categorical)[train_idx],
                        epochs=10, batch_size=32,
                        validation_data=(np.array(X_train_img)[val_idx], np.array(y_train_categorical)[val_idx]))
    
    # Evaluierung auf dem Validierungsset
    score = model.evaluate(np.array(X_train_img)[val_idx], np.array(y_train_categorical)[val_idx], verbose=0)
    print(f"Fold {fold_no} - Validation Loss: {score[0]} - Validation Accuracy: {score[1]}")
    scores.append(score[1])
    
    fold_no += 1

# Ausgabe der mittleren und Standardabweichung der Validierungsgenauigkeit
print(f"Durchschnittliche Validierungsgenauigkeit: {np.mean(scores)}")
print(f"Standardabweichung der Validierungsgenauigkeiten: {np.std(scores)}")

Training für fold 1...
Epoch 1/10
17/17 ━━━━━━━━━━━━━━━━━━━━ 14s 740ms/step - accuracy: 0.2306 - loss: 2.0073 - val_accuracy: 0.2529 - val_loss: 1.6220
Epoch 2/10
17/17 ━━━━━━━━━━━━━━━━━━━━ 12s 706ms/step - accuracy: 0.3310 - loss: 1.4702 - val_accuracy: 0.5136 - val_loss: 1.1527
Epoch 3/10
17/17 ━━━━━━━━━━━━━━━━━━━━ 13s 748ms/step - accuracy: 0.5317 - loss: 1.0711 - val_accuracy: 0.5253 - val_loss: 1.0967
Epoch 4/10
17/17 ━━━━━━━━━━━━━━━━━━━━ 21s 797ms/step - accuracy: 0.5728 - loss: 1.0258 - val_accuracy: 0.5914 - val_loss: 1.0696
Epoch 5/10
17/17 ━━━━━━━━━━━━━━━━━━━━ 19s 721ms/step - accuracy: 0.6456 - loss: 0.9018 - val_accuracy: 0.6654 - val_loss: 0.8548
Epoch 6/10
17/17 ━━━━━━━━━━━━━━━━━━━━ 21s 719ms/step - accuracy: 0.7566 - loss: 0.6560 - val_accuracy: 0.5681 - val_loss: 1.1016
Epoch 7/10
17/17 ━━━━━━━━━━━━━━━━━━━━ 12s 710ms/step - accuracy: 0.7404 - loss: 0.6336 - val_accuracy: 0.5992 - val_loss: 1.0316
Epoch 8/10
17/17 ━━━━━━━━━━━━━━━━━━━━ 13s 756ms/step - accuracy: 0.8091 - 